# Import Libraries

In [20]:
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException
import json
import os
import pandas as pd
from langdetect import detect
import random
import re
import time
from sklearn.model_selection import train_test_split
import ast

# Reading the file

In [2]:
#lockdown1 Duration- 20th March to 14th April
df=pd.read_csv("covid2.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4548 entries, 0 to 4547
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   User       4548 non-null   object
 1   Text       4548 non-null   object
 2   Date       4548 non-null   object
 3   Retweets   4548 non-null   int64 
 4   Favorites  4548 non-null   int64 
 5   HashTags   4547 non-null   object
 6   City       4548 non-null   object
dtypes: int64(2), object(5)
memory usage: 160.0+ KB


,User,Text,Date,Retweets,Favorites,HashTags,City
0,NikhilMRoy,I am pleased to share that I have passed and c...,2020-05-02 23:42:40+00:00,0,1,#weekenddoneright #PSM #scrum #scrummaster #ac...,Mumbai
1,satraparth,#COVID19 lockdown has affected various sectors...,2020-05-02 20:41:13+00:00,0,0,#COVID19 #pandemic #MSME #lockdown,Mumbai
2,iampawannathani,"#covid19 will need to be fought through, it ca...",2020-05-02 19:08:02+00:00,0,1,#covid19 #Warzone #FightAgainstCorona,Mumbai
3,iampawannathani,#Lockdown3 @PMOIndia If #COVID19 is here to st...,2020-05-02 19:07:59+00:00,1,3,#Lockdown3 #COVID19 #India,Mumbai
4,sirajnoorani,#COVID19 UP CM Yogi Adityanath directs officia...,2020-05-02 19:05:14+00:00,0,0,#COVID19,Mumbai


# Cleaning

In [3]:
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [4]:
df['clean_tweet'] = df['Text'].apply(lambda x: clean_tweet(x))
df.head()
df.to_csv('covidclean1.csv',index=False)

In [5]:
data = pd.read_csv('covidclean1.csv')
data= data.iloc[0:2001,]
data


,User,Text,Date,Retweets,Favorites,HashTags,City,clean_tweet
0,NikhilMRoy,I am pleased to share that I have passed and c...,2020-05-02 23:42:40+00:00,0,1,#weekenddoneright #PSM #scrum #scrummaster #ac...,Mumbai,I am pleased to share that I have passed and c...
1,satraparth,#COVID19 lockdown has affected various sectors...,2020-05-02 20:41:13+00:00,0,0,#COVID19 #pandemic #MSME #lockdown,Mumbai,COVID19 lockdown has affected various sectors ...
2,iampawannathani,"#covid19 will need to be fought through, it ca...",2020-05-02 19:08:02+00:00,0,1,#covid19 #Warzone #FightAgainstCorona,Mumbai,covid19 will need to be fought through it cann...
3,iampawannathani,#Lockdown3 @PMOIndia If #COVID19 is here to st...,2020-05-02 19:07:59+00:00,1,3,#Lockdown3 #COVID19 #India,Mumbai,Lockdown3 If COVID19 is here to stay for long ...
4,sirajnoorani,#COVID19 UP CM Yogi Adityanath directs officia...,2020-05-02 19:05:14+00:00,0,0,#COVID19,Mumbai,COVID19 UP CM Yogi Adityanath directs official...
...,...,...,...,...,...,...,...,...
1996,aanondo,"If data is generated, consumed and recorded wi...",2020-04-20 15:26:17+00:00,0,0,#COVID19,Delhi,If data is generated consumed and recorded wit...
1997,sidhant,First ever e-@ITECnetwork training programme o...,2020-04-20 15:26:02+00:00,8,19,#COVID19,Delhi,First ever e training programme on COVID19 Man...
1998,SBM_4007,#COVID19 update: India crosses 3000 recoveries...,2020-04-20 15:24:43+00:00,3,4,#COVID19 #IndiaFightsCorona,Delhi,COVID19 update India crosses 3000 recoveries F...
1999,HealthwireMedia,#WUD Takes Initiative In Fight Against #COVID1...,2020-04-20 15:15:09+00:00,3,4,#WUD #COVID19 #PPE,Delhi,WUD Takes Initiative In Fight Against COVID19 ...


# IBM Tone Analyser

In [21]:
authenticator = IAMAuthenticator('iY-jyXKEwdMyEJvLigD3kRGUoN2ZNjjqwOUp0pxOzuQu')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.eu-gb.tone-analyzer.watson.cloud.ibm.com/instances/416622a0-5faa-4efd-8868-a0318d2167bd')
tone_analyzer.set_disable_ssl_verification(False)

In [ ]:
tones = []
index = 0
for item in data['clean_tweet']:
    try:
        index = index + 1
        text = item
        tone_analysis = tone_analyzer.tone({'text': text}, content_type='application/json').get_result()
        tones.append(tone_analysis)
    except ApiException as ex:
        tones.append(None)
        print ("Method failed with status code " + str(ex.code) + ": " + ex.message)

In [ ]:
tones_copy = tones.copy()
tones_copy = pd.DataFrame(tones_copy)

In [ ]:
tones_copy.head()

In [ ]:
data = data.reset_index()

In [ ]:
data = data.drop(['index'], axis=1)

In [ ]:
data = pd.concat([data,tones_copy],axis=1)
data.to_csv('covid2_4000_lock2.csv',index=False)

# Formatting the document tones

In [ ]:
data = pd.read_csv('covid2_4000_lock2.csv')
data.head()

In [ ]:
data = data.iloc[:,1:26]

In [ ]:
data['document_tones'] = data.document_tone.apply(lambda x: ast.literal_eval(x)['tones'])
data.drop('document_tone',axis = 1,inplace = True)

In [ ]:
# split sentiments in corresponding columns
for i in range(0, len(data)):
    for j in range(0, len(data.loc[i, 'document_tones'])):
        dic = data.loc[i, 'document_tones'][j]
        source = dic['tone_id']
        data.loc[i, source] = 1
data.head()

In [ ]:

data.to_csv('covid2_2000_lock2_sentiment.csv',index=False)

# First 500 sentiments 
## Analytical ,joy, sadness, tentative, confident, anger, fear (Lockdown 1)

In [ ]:
data

# Combining all datasets with tones analyzed by IBM 

In [19]:
path = r'E:\COVID\Twitter Data for IBM\clean sentiments IBM\\'
files = os.listdir(path)

In [22]:
files

['covid1_1000_lock1_sentiment.csv',
 'covid1_1500_lock1_sentiment.csv',
 'covid1_2500_lock1_sentiment.csv',
 'covid1_4500_lock1_sentiment.csv',
 'covid1_500_lock1_sentiment.csv',
 'covid1_5500_lock1_sentiment.csv',
 'covid1_7500_lock1sentiment.csv',
 'covid1_last_lock1sentiment.csv',
 'covid2_2000_lock2_sentiment.csv',
 'covid2_4000_lock2_sentiment.csv',
 'covid2_4549_lock2_sentiment.csv',
 'covid3_1724_lock3_sentiment.csv',
 'covid3_3451_lock3_sentiment.csv',
 'covid4_1103_lock4_sentiment.csv',
 'covid4_2207_lock4_sentiment.csv']

In [23]:
covid_twitter_data = pd.DataFrame()
# Concat the Twitters data into one-table
for file in files:
    try:
        data = pd.read_csv(str(path) + file)
    except:
        data = pd.read_csv(str(path) + file,encoding='cp1252')
    covid_twitter_data = covid_twitter_data.append(data, ignore_index=True)

In [24]:
covid_twitter_data['fear'].sum()

435.0

In [25]:
# Save the dataset
covid_twitter_data.to_csv('sentimentIBMfull.csv',index=False)


In [26]:
covid_twitter_data.head()

,Text,Date,Retweets,Favorites,HashTags,City,clean_tweet,document_tones,joy,sadness,confident,analytical,tentative,anger,fear
0,"Stone “Chilled” Steve Austin, is in the house....",2020-04-04 10:23:04+00:00,3,12,#stonecoldsteveaustin #lockdown #meme #fun #co...,Mumbai,Stone Chilled Steve Austin is in the house No ...,"[{'score': 0.844069, 'tone_id': 'joy', 'tone_n...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Long run was must do activity in weekends for ...,2020-04-04 08:58:48+00:00,0,4,#lockdown #covid19 #fit4life,Mumbai,Long run was must do activity in weekends for ...,"[{'score': 0.584247, 'tone_id': 'joy', 'tone_n...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,A 51-year-old #COVID19 positive patient who at...,2020-04-04 08:33:10+00:00,0,1,#COVID19,Mumbai,A 51 year old COVID19 positive patient who att...,"[{'score': 0.708308, 'tone_id': 'sadness', 'to...",NaN,1.0,1.0,NaN,NaN,NaN,NaN
3,We are going through a tough time. It is very ...,2020-04-04 08:31:33+00:00,1,1,#Coronavirus #StayStrong #Covid19 #PMCaresFund,Mumbai,We are going through a tough time It is very c...,"[{'score': 0.605143, 'tone_id': 'confident', '...",NaN,NaN,1.0,NaN,NaN,NaN,NaN
4,#DeepikaPadukone and #RanveerSingh to contribu...,2020-04-04 08:27:33+00:00,0,1,#DeepikaPadukone #RanveerSingh #PMCaresFund #C...,Mumbai,DeepikaPadukone and RanveerSingh to contribute...,"[{'score': 0.620279, 'tone_id': 'analytical', ...",NaN,NaN,NaN,1.0,NaN,NaN,NaN
